<a href="https://colab.research.google.com/github/twosegfaults/C/blob/master/Chatbot_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets
!pip install transformers[torch]
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.7 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 2

In [4]:
import pandas as pd
import json

df = pd.read_csv('medquad.csv')
df_dict = df.to_dict('records')

train = df_dict[:10000]
test = df_dict[10000:]

def write_to_file(data, output_file):
        with open(output_file, 'w') as outfile:
            for entry in data:
                text = f"Question: {entry['question']}. Answer: {entry['answer']}."
                json_line = json.dumps({"text": text})
                outfile.write(json_line + '\n')

write_to_file(train, "train.jsonl")
write_to_file(test, "test.jsonl")

In [5]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

dataset_train = load_dataset('json', data_files='train.jsonl')
dataset_test = load_dataset('json', data_files='test.jsonl')

tokenizer = AutoTokenizer.from_pretrained("GG2k4/GPT2")
tokenizer.pad_token = tokenizer.eos_token

def tokenize_function(examples):
    tokenized_inputs = tokenizer(examples['text'], padding="max_length", truncation=True, max_length=1000)
    tokenized_inputs['labels'] = tokenized_inputs['input_ids']
    return tokenized_inputs

tokenized_datasets_train = dataset_train.map(tokenize_function, batched=True)
tokenized_datasets_test = dataset_test.map(tokenize_function, batched=True)

model = AutoModelForCausalLM.from_pretrained("GG2k4/GPT2")
if torch.cuda.is_available():
    model.to('cuda')
else:
    model.to('cpu')

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/667 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

...

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/6412 [00:00<?, ? examples/s]

config.json:   0%|          | 0.00/918 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

In [7]:
def compute_loss(model, inputs):
    labels = inputs.pop('labels')  # Remove labels from inputs
    outputs = model(**inputs)
    logits = outputs.logits
    loss_fn = torch.nn.CrossEntropyLoss()
    loss = loss_fn(logits.view(-1, logits.size(-1)), labels.view(-1))
    return loss

training_args = TrainingArguments(
    output_dir='./results',
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=1,
    save_steps=0,
    save_total_limit=2,
    logging_dir='./logs',
    logging_steps=500,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets_train['train'],
    eval_dataset=tokenized_datasets_test,
    compute_metrics=compute_loss,
)

trainer.train()

model.save_pretrained('./fine-tuned-gpt2')
tokenizer.save_pretrained('./fine-tuned-gpt2')

Step,Training Loss
500,0.643700
1000,0.521800
1500,0.552200
2000,0.500500
2500,0.508800
3000,0.483900
3500,0.486900
4000,0.484100
4500,0.489800
5000,0.469500


('./fine-tuned-gpt2/tokenizer_config.json',
 './fine-tuned-gpt2/special_tokens_map.json',
 './fine-tuned-gpt2/vocab.json',
 './fine-tuned-gpt2/merges.txt',
 './fine-tuned-gpt2/added_tokens.json',
 './fine-tuned-gpt2/tokenizer.json')

In [ ]:
from transformers import pipeline

fine_tuned_model = GPT2LMHeadModel.from_pretrained('./fine-tuned-gpt2')
fine_tuned_tokenizer = GPT2Tokenizer.from_pretrained('./fine-tuned-gpt2')

generator = pipeline('text-generation', model=fine_tuned_model, tokenizer=fine_tuned_tokenizer)


In [ ]:
with open('coverage_eval_qa.json', 'r') as f:
  coverage_qa = json.load(f)
with open('medical_terms_qa.json', 'r') as f:
  medical_qa = json.load(f)
with open('insurance_eval_qa.json', 'r') as f:
  insurance_qa = json.load(f)

qa_pairs = insurance_qa['questions'] + coverage_qa['questions'] + medical_qa['questions']


In [ ]:
import numpy as np
import time
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from difflib import SequenceMatcher

def extract_definition(input_string):
    try:
        after_definition = input_string.split('?')[1]
        if 'Definition:' in after_definition:
            after_definition = after_definition.split('Definition:')[1]
        if 'Examples:' in after_definition:
            after_definition = after_definition.split('Examples:')[0]
        if 'Category:' in after_definition:
            after_definition = after_definition.split('Category:')[0]
        return after_definition.strip()
    except IndexError:
        return input_string

def cosine_sim(a, b):
    vectorizer = TfidfVectorizer().fit_transform([a, b])
    vectors = vectorizer.toarray()
    return cosine_similarity(vectors)[0, 1]

def exact_match(a, b):
    return SequenceMatcher(None, a, b).ratio()

def get_preds_and_metrics(generator, data):
    predictions= []
    cosine_similarities = []
    exact_match_scores = []
    response_times = []
    for qa in data:
      question = data[0]['question']
      true_ans = data[0]['answer']
      start_time = time.time()
      response = generator(question, max_length=50, num_return_sequences=1, truncation = True)[0]['generated_text']
      result = extract_definition(response)
      end_time = time.time()
      response_times.append(end_time - start_time)
      predictions.append(result)
      cosine_score = cosine_sim(result, true_ans)
      exact_match_score = exact_match(result, true_ans)
      cosine_similarities.append(cosine_score)
      exact_match_scores.append(exact_match_score)
    return predictions, response_times, cosine_similarities, exact_match_scores



preds, rt, csims, emscores = get_preds_and_metrics(generator, qa_pairs)
average_csim = sum(csims)/len(csims)
average_emscores = sum(emscores)/len(emscores)
average_rt = sum(rt)/len(rt)

print(f"Average Cosine Similarity: {average_csim}")
print(f"Average Exact Match Score: {average_emscores}")
print(f"Average Response Time: {average_rt} seconds")

Average Cosine Similarity: 0.5953495487131456
Average Exact Match Score: 0.6045980665091628
Average Response Time: 1.0021773195266723 seconds


In [8]:
from huggingface_hub import HfApi, HfFolder, Repository, notebook_login

notebook_login()

In [10]:
model = AutoModelForCausalLM.from_pretrained("fine-tuned-gpt2")
tokenizer.save_pretrained("GPT2")
trainer.create_model_card()
trainer.push_to_hub("twosegfaults/GPT2_2")
tokenizer.push_to_hub("twosegfaults/GPT2_2")
model.push_to_hub("twosegfaults/GPT2_2")

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/twosegfaults/GPT2_2/commit/e901eae6f66033089a2c1dc49b38eaf7d4cb2432', commit_message='Upload model', commit_description='', oid='e901eae6f66033089a2c1dc49b38eaf7d4cb2432', pr_url=None, pr_revision=None, pr_num=None)

In [22]:
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForCausalLM
def extract_definition(input_string):
    try:
        after_definition = input_string.split('')[1]
        if 'Definition:' in after_definition:
            after_definition = after_definition.split('Definition:')[1]
        if 'Examples:' in after_definition:
            after_definition = after_definition.split('Examples:')[0]
        if 'Category:' in after_definition:
            after_definition = after_definition.split('Category:')[0]
        return after_definition.strip()
    except IndexError:
        return input_string
model = AutoModelForCausalLM.from_pretrained("GG2k4/GPT2")
tokenizer = AutoTokenizer.from_pretrained("GG2k4/GPT2")
generator = pipeline('text-generation', model=model, tokenizer=tokenizer)
response = generator("What is cancer?", max_length=1000, num_return_sequences=1, truncation = False)[0]['generated_text']
print(response)

What is cancer? Definition: A disease that occurs when your cells get too big and swell and may produce too many cells..
